# Sentiment Analysis Using NLP
A sentiment analysis model was applied to the IMDB dataset with two implementations: one using Bag of Words (BoW) and the other using TF-IDF. The BoW implementation achieved an accuracy of 0.863, while the TF-IDF implementation achieved a slightly lower accuracy of 0.857. Both models were further tested on real-world reviews: a review of The Middle by Dustin Rowles and a review of The Emoji Movie by Leigh Monson. In both cases, the models correctly identified the sentiment, labeling the review for The Middle as positive and the review for The Emoji Movie as negative. These results demonstrate that both feature extraction methods are effective for sentiment classification, with BoW showing a marginally higher overall accuracy.

Sources:

Tossed Salad and Scrambled Eggs by Dustin Rowles:
https://www.pajiba.com/tv_reviews/the-middle-review.php

‘The Emoji Movie’ is an ersatz film built on a foundation of garbage by Leigh Monson: https://substreammagazine.com/2017/08/emoji-movie-review/

# Sentiment Analysis Using NLP: Bag-of-Words (BoW)

**Load IMDB Dataset**

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load dataset (keeping top 5000 words)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=5000)

# Pad sequences to make them same length
x_train = pad_sequences(x_train, maxlen=500)
x_test = pad_sequences(x_test, maxlen=500)


**Decode IMDB Tokenized Data Into Actual Text**

In [ ]:
word_index = imdb.get_word_index()
reverse_index = {value: key for (key, value) in word_index.items()}

def decode_review(encoded):
    return " ".join([reverse_index.get(i - 3, "?") for i in encoded])

texts_train = [decode_review(x) for x in x_train]
texts_test = [decode_review(x) for x in x_test]



**Vectorize Texts Using BoW**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer(max_features=5000)
X_train_bow = bow.fit_transform(texts_train).toarray()
X_test_bow = bow.transform(texts_test).toarray()


**Build**

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_bow.shape[1],)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Train**

In [ ]:
model.fit(X_train_bow, y_train, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.8165 - loss: 0.4077 - val_accuracy: 0.8764 - val_loss: 0.3143
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9219 - loss: 0.1994 - val_accuracy: 0.8770 - val_loss: 0.3089
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9568 - loss: 0.1240 - val_accuracy: 0.8662 - val_loss: 0.3480
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9862 - loss: 0.0554 - val_accuracy: 0.8768 - val_loss: 0.3905
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9982 - loss: 0.0181 - val_accuracy: 0.8752 - val_loss: 0.4973


**Evaluate**

In [ ]:
loss, acc = model.evaluate(X_test_bow, y_test)
print("Accuracy:", acc)

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8647 - loss: 0.5147
Accuracy: 0.8629599809646606


**Test Model On Actual Reviews**

In [ ]:
import numpy as np

def predict_sentiment(text):
    # Preprocess the input text using the same bow vectorizer
    processed_text = bow.transform([text]).toarray()

    # Make a prediction using the trained model
    prediction = model.predict(processed_text)

    # Interpret the prediction
    if prediction[0] > 0.5:
        return "Positive", prediction[0][0]
    else:
        return "Negative", prediction[0][0]

# Example usage:
input_text = "The first two weeks of the Fall season are finally coming to a close, and most of the premieres have rolled out. We’ve still got a few reviews of pilots coming, before we turn to reviewing some of the returning shows, but I’ll be glad to rid my DVR of some of the more unwatchable premieres and settle into a nice weekly television schedule. Last night, the other half of ABC’s Wednesday night comedy block debuted (supplementing last week’s debuts of the more superior shows, “Modern Family” and to a much lesser extent, “Cougar Town”). The first, “Hank,” is an easy one to delete from the DVR — in fact, I nearly did so before the pilot had finished. It’s woefully, painfully unwatchable. The premise itself suits its star, Kelsey Grammar, but that is the only nice thing I can muster to say about “Hank.” It’s about a wealthy blowhard CEO in Manhattan who gets downsized and is forced to give up his nannies and maids and move to a much smaller, humbler house in Virginia, where he has to actually spend time with his family. “Hank” is another traditional, laugh-track sitcom, and I can’t for the life of me figure out why the networks continue to try. There are a couple of successful ones on CBS (“Two and a Half Men,” and “Big Bang Theory”) but other than those two, I can’t think of a conventional sitcom that’s survived more than two years since “Friends” and “Fraiser” went off the air. Even worse, it relies on setups and bad jokes, instead of situational humor — as though real people go around cracking wise and making bad puns at every opportunity. Misunderstandings, slips-of-the-tongue, and telegraphed punch-lines dominate the show, which is terribly overacted, hammy, and completely wastes the talents of Grammer, who is a much better actor than his last two shows have indicated. Although, the show did get me thinking about what might be an appropriate role for Grammer, and I figured it out: He and — for a little meta-ironical fun — Dan Aykroyd should pair up as Winthrop and Mortimer in a remake of Trading Places. They’re kind of perfect for it. Patricia Heaton’s new show, “The Middle,” unfortunately, isn’t as easy to dismiss. I really wanted to dislike it more than I did, because (for political reasons) I want Heaton to die in a grease fire. “The Middle” is sort of a cross between “Malcolm in the Middle” and “Roseanne,” and while it succeeds, occasionally, as a more modern “Roseanne,” the “Malcolm” aspects of the show don’t work as well. Heaton plays Frankie, the matriarch of a lower-middle class family in Flyover Country (Indiana). She’s struggling to make enough money as a used-car salesman to pay for the gas to get to and from her job, while also dealing with motherhood. She has three modestly dysfunctional children: Axl, her 15-year-old sarcastic son, who walks around the house in his boxers and grunts occasionally; Sue, her awkward teenager daughter, who has yet to find any sort of talent in school; and Brick, her youngest, who lacks basic social skills and talks to his backpack. (The reason for the names “Brick” and “Axl” is that the parents had hoped, by giving them a “cool” name, that they’d turn out to be “cool” children. The plan clearly backfired). Then there’s Mike, her husband (Neil Flynn), who offers enough reason to watch a few more episodes to see where “The Middle” is going — he’s a wryly funny as he was as the Janitor of “Scrubs,” though he’s given a little more substance, more warmth, and even a name. Where “The Middle” succeeds, occasionally, is where it attempts to satirize Middle American values while also embracing them. It’s a difficult line to toe, and with Heaton doing her best Jane Kaczmarek, it’s often hard to tell how much the show is making fun of Flyover country and how much it’s celebrating it. That’s actually to the show’s credit, as it manages to appeal to Middle Americans, cultural elitist, and even those like myself, former Middle Americans who have relocated but are still able to warmly identify with fast-food family dinners surrounding the television set. “The Middle,” appropriately enough, is not nearly as good as “Modern Family,” nor is it nearly as bad as the other show that bookends it, “Hank.” It’s somewhere in the middle. And there’s just enough promise to keep a season pass on my DVR a few more weeks to see where it’s going. ."
sentiment, probability = predict_sentiment(input_text)
print(f"Input: '{input_text}'")
print(f"Predicted Sentiment: {sentiment} (Probability: {probability:.4f})")

input_text = "Allow me recount to you the two instances where I laughed during The Emoji Movie. The first is as our heroes encounter the Just Dance app, and the big smiling avatar of that game encourages them to dance their way through to the other side of a chasm. When their playing activates sounds and displays on the phone’s interface, the phone’s owner deletes the app in frustration. This causes the app to collapse around the emojis, including the game’s mascot as she writhes in the most unimaginable pain of being erased from existence. The second time I laughed was when we visit the phone’s trash, and we see that same mascot curled in a ball on a garbage pile, mascara running down her face, muttering to herself that she just wants to dance as she awaits her final, permanent death. These are not scenes played for comedy, but they stand in such stark, tone-deaf contrast to the rest of the film’s bright, friendly aesthetic that it’s hard not to find a sick sense of schadenfreude in what is an otherwise tediously boring film. I apologize to the children at my screening for the sudden outbursts. Ostensibly, The Emoji Movie is the story of three emojis traveling from the messenger app on their phone to the mythical Cloud so that they may each get something to feel more complete. The inexplicably named Gene (T.J. Miller) is a “meh” emoji who has an equally inexplicable ability to make other faces than the one assigned to him and wishes only to be normal; Hi-5 (James Corden) is a gross anthropomorphized hand who wants nothing more than to be hacked back into the phone’s favorites category; and Jailbreak (Anna Faris) is a hacker with a mysterious past who promises to help the pair in exchange for using Gene’s face-changing ability to get past a firewall into the Cloud. The lessons of this story are as inevitable as they are hackneyed, though they operate on a weird double standard. Gene’s arc culminates with the confidence to express the wide range of emotions afforded him, whereas Jailbreak discovers that her assigned role within emoji society is right for her and that her adopted persona as a hacker is ingenuine to her true self. This all comes across as cloyingly ill-considered as Jailbreak’s dialogue insistently inserts faux-feminist representation rhetoric, only to have the all-is-lost moment of the script come as she rejects Gene’s romantic advances, causing Gene to lose his unique expressiveness so that Jailbreak can return to him in the third act out of… guilt, I suppose? Consciously sexist it doesn’t appear to be, but subtextually it is problematic as all get-out that the female lead gives up her autonomy to be a supporting player for the male lead as he discovers his own.Alas, the purpose of The Emoji Movie is not to be an enlightened take on gender parity, and the fact that it tells a barely coherent story where the mechanics of the world within the smartphone are constantly suspect only tokenly masks the film’s true raison d’etre: product placement. Real world apps litter the scenes of this film, from YouTube to Facebook to the aforementioned Just Dance. When you get right down to it, there is little to The Emoji Movie beyond acting as a walking tour of the wonderous world of social media brand management, shining logos in front of kiddies with the intent to indoctrinate them into accepting their platforms of choice as wonderlands where their emoji friends can travel for theirs and the children’s amusement. It would be insidious were it not so ineptly obvious, and many a parent is going to regret succumbing to their child’s demands to purchase a ticket, for the film in turn encourages further consumerist indoctrination. I wish I could at least say that some of the film’s many attempts at humor land, but uniformly the writers pick the most pathetically obvious puns about technology and poop—the latter of which are largely delivered by Patrick Stewart of all people, whose character Poop is once seen in a chair shouting “red alert”—and the delivery of every single one of these lines sounds so flat and bored that it’s clear that almost nobody cared to save the film from itself. The only one putting in any effort seems to be James Corden, whose vocal work is as bouncy as his character, but doesn’t have the timing to make any of his cringe-worthy lines work. If there’s any image that sums up the movie perfectly, it’s of that Just Dance avatar curled up and crying in the dump. It’s a horrifying scene set atop a pile of trash, but the pure absurdity of its existence is enough to make one laugh in shock and disbelief, at least in that one moment. The symbolism of that moment will stick with me far more than the stale and stilted messages The Emoji Movie intended.."
sentiment, probability = predict_sentiment(input_text)
print(f"\nInput: '{input_text}'")
print(f"Predicted Sentiment: {sentiment} (Probability: {probability:.4f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Input: 'The first two weeks of the Fall season are finally coming to a close, and most of the premieres have rolled out. We’ve still got a few reviews of pilots coming, before we turn to reviewing some of the returning shows, but I’ll be glad to rid my DVR of some of the more unwatchable premieres and settle into a nice weekly television schedule. Last night, the other half of ABC’s Wednesday night comedy block debuted (supplementing last week’s debuts of the more superior shows, “Modern Family” and to a much lesser extent, “Cougar Town”). The first, “Hank,” is an easy one to delete from the DVR — in fact, I nearly did so before the pilot had finished. It’s woefully, painfully unwatchable. The premise itself suits its star, Kelsey Grammar, but that is the only nice thing I can muster to say about “Hank.” It’s about a wealthy blowhard CEO in Manhattan who gets downsized and is forced to give up his nannies and maids and move to a much smaller, humbl

# Sentiment Analysis Using NLP: TF-IDF

**Load IMDB Dataset**

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load dataset (keeping top 5000 words)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=5000)

# Pad sequences to make them same length
x_train = pad_sequences(x_train, maxlen=500)
x_test = pad_sequences(x_test, maxlen=500)

**Decode IMDB Tokenized Data Into Actual Text**

In [ ]:
word_index = imdb.get_word_index()
reverse_index = {value: key for (key, value) in word_index.items()}

def decode_review(encoded):
    return " ".join([reverse_index.get(i - 3, "?") for i in encoded])

texts_train = [decode_review(x) for x in x_train]
texts_test = [decode_review(x) for x in x_test]

**Vectorize Texts Using TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(texts_train).toarray()
X_test_tfidf = tfidf_vectorizer.transform(texts_test).toarray()

**Build**

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_tfidf.shape[1],)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Train**

In [ ]:
model.fit(X_train_tfidf, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.8103 - loss: 0.4643 - val_accuracy: 0.8860 - val_loss: 0.2713
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9196 - loss: 0.2149 - val_accuracy: 0.8808 - val_loss: 0.2863
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9391 - loss: 0.1685 - val_accuracy: 0.8748 - val_loss: 0.3185
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9439 - loss: 0.1489 - val_accuracy: 0.8748 - val_loss: 0.3335
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9533 - loss: 0.1277 - val_accuracy: 0.8696 - val_loss: 0.3642


**Evaluate**

In [ ]:
loss, acc = model.evaluate(X_test_tfidf, y_test)
print(f"Accuracy: {acc}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8593 - loss: 0.3873
Accuracy: 0.8571199774742126


**Test Model On Actual Reviews**

In [ ]:
import numpy as np

def predict_sentiment(text):
    # Preprocess the input text using the tfidf_vectorizer
    processed_text = tfidf_vectorizer.transform([text]).toarray()

    # Make a prediction using the trained model
    prediction = model.predict(processed_text)

    # Interpret the prediction
    if prediction[0] > 0.5:
        return "Positive", prediction[0][0]
    else:
        return "Negative", prediction[0][0]

# Example usage:
input_text = "The first two weeks of the Fall season are finally coming to a close, and most of the premieres have rolled out. We’ve still got a few reviews of pilots coming, before we turn to reviewing some of the returning shows, but I’ll be glad to rid my DVR of some of the more unwatchable premieres and settle into a nice weekly television schedule. Last night, the other half of ABC’s Wednesday night comedy block debuted (supplementing last week’s debuts of the more superior shows, “Modern Family” and to a much lesser extent, “Cougar Town”). The first, “Hank,” is an easy one to delete from the DVR — in fact, I nearly did so before the pilot had finished. It’s woefully, painfully unwatchable. The premise itself suits its star, Kelsey Grammar, but that is the only nice thing I can muster to say about “Hank.” It’s about a wealthy blowhard CEO in Manhattan who gets downsized and is forced to give up his nannies and maids and move to a much smaller, humbler house in Virginia, where he has to actually spend time with his family. “Hank” is another traditional, laugh-track sitcom, and I can’t for the life of me figure out why the networks continue to try. There are a couple of successful ones on CBS (“Two and a Half Men,” and “Big Bang Theory”) but other than those two, I can’t think of a conventional sitcom that’s survived more than two years since “Friends” and “Fraiser” went off the air. Even worse, it relies on setups and bad jokes, instead of situational humor — as though real people go around cracking wise and making bad puns at every opportunity. Misunderstandings, slips-of-the-tongue, and telegraphed punch-lines dominate the show, which is terribly overacted, hammy, and completely wastes the talents of Grammer, who is a much better actor than his last two shows have indicated. Although, the show did get me thinking about what might be an appropriate role for Grammer, and I figured it out: He and — for a little meta-ironical fun — Dan Aykroyd should pair up as Winthrop and Mortimer in a remake of Trading Places. They’re kind of perfect for it. Patricia Heaton’s new show, “The Middle,” unfortunately, isn’t as easy to dismiss. I really wanted to dislike it more than I did, because (for political reasons) I want Heaton to die in a grease fire. “The Middle” is sort of a cross between “Malcolm in the Middle” and “Roseanne,” and while it succeeds, occasionally, as a more modern “Roseanne,” the “Malcolm” aspects of the show don’t work as well. Heaton plays Frankie, the matriarch of a lower-middle class family in Flyover Country (Indiana). She’s struggling to make enough money as a used-car salesman to pay for the gas to get to and from her job, while also dealing with motherhood. She has three modestly dysfunctional children: Axl, her 15-year-old sarcastic son, who walks around the house in his boxers and grunts occasionally; Sue, her awkward teenager daughter, who has yet to find any sort of talent in school; and Brick, her youngest, who lacks basic social skills and talks to his backpack. (The reason for the names “Brick” and “Axl” is that the parents had hoped, by giving them a “cool” name, that they’d turn out to be “cool” children. The plan clearly backfired). Then there’s Mike, her husband (Neil Flynn), who offers enough reason to watch a few more episodes to see where “The Middle” is going — he’s a wryly funny as he was as the Janitor of “Scrubs,” though he’s given a little more substance, more warmth, and even a name. Where “The Middle” succeeds, occasionally, is where it attempts to satirize Middle American values while also embracing them. It’s a difficult line to toe, and with Heaton doing her best Jane Kaczmarek, it’s often hard to tell how much the show is making fun of Flyover country and how much it’s celebrating it. That’s actually to the show’s credit, as it manages to appeal to Middle Americans, cultural elitist, and even those like myself, former Middle Americans who have relocated but are still able to warmly identify with fast-food family dinners surrounding the television set. “The Middle,” appropriately enough, is not nearly as good as “Modern Family,” nor is it nearly as bad as the other show that bookends it, “Hank.” It’s somewhere in the middle. And there’s just enough promise to keep a season pass on my DVR a few more weeks to see where it’s going. ."
sentiment, probability = predict_sentiment(input_text)
print(f"Input: '{input_text}'")
print(f"Predicted Sentiment: {sentiment} (Probability: {probability:.4f})")

input_text = "Allow me recount to you the two instances where I laughed during The Emoji Movie. The first is as our heroes encounter the Just Dance app, and the big smiling avatar of that game encourages them to dance their way through to the other side of a chasm. When their playing activates sounds and displays on the phone’s interface, the phone’s owner deletes the app in frustration. This causes the app to collapse around the emojis, including the game’s mascot as she writhes in the most unimaginable pain of being erased from existence. The second time I laughed was when we visit the phone’s trash, and we see that same mascot curled in a ball on a garbage pile, mascara running down her face, muttering to herself that she just wants to dance as she awaits her final, permanent death. These are not scenes played for comedy, but they stand in such stark, tone-deaf contrast to the rest of the film’s bright, friendly aesthetic that it’s hard not to find a sick sense of schadenfreude in what is an otherwise tediously boring film. I apologize to the children at my screening for the sudden outbursts. Ostensibly, The Emoji Movie is the story of three emojis traveling from the messenger app on their phone to the mythical Cloud so that they may each get something to feel more complete. The inexplicably named Gene (T.J. Miller) is a “meh” emoji who has an equally inexplicable ability to make other faces than the one assigned to him and wishes only to be normal; Hi-5 (James Corden) is a gross anthropomorphized hand who wants nothing more than to be hacked back into the phone’s favorites category; and Jailbreak (Anna Faris) is a hacker with a mysterious past who promises to help the pair in exchange for using Gene’s face-changing ability to get past a firewall into the Cloud. The lessons of this story are as inevitable as they are hackneyed, though they operate on a weird double standard. Gene’s arc culminates with the confidence to express the wide range of emotions afforded him, whereas Jailbreak discovers that her assigned role within emoji society is right for her and that her adopted persona as a hacker is ingenuine to her true self. This all comes across as cloyingly ill-considered as Jailbreak’s dialogue insistently inserts faux-feminist representation rhetoric, only to have the all-is-lost moment of the script come as she rejects Gene’s romantic advances, causing Gene to lose his unique expressiveness so that Jailbreak can return to him in the third act out of… guilt, I suppose? Consciously sexist it doesn’t appear to be, but subtextually it is problematic as all get-out that the female lead gives up her autonomy to be a supporting player for the male lead as he discovers his own.Alas, the purpose of The Emoji Movie is not to be an enlightened take on gender parity, and the fact that it tells a barely coherent story where the mechanics of the world within the smartphone are constantly suspect only tokenly masks the film’s true raison d’etre: product placement. Real world apps litter the scenes of this film, from YouTube to Facebook to the aforementioned Just Dance. When you get right down to it, there is little to The Emoji Movie beyond acting as a walking tour of the wonderous world of social media brand management, shining logos in front of kiddies with the intent to indoctrinate them into accepting their platforms of choice as wonderlands where their emoji friends can travel for theirs and the children’s amusement. It would be insidious were it not so ineptly obvious, and many a parent is going to regret succumbing to their child’s demands to purchase a ticket, for the film in turn encourages further consumerist indoctrination. I wish I could at least say that some of the film’s many attempts at humor land, but uniformly the writers pick the most pathetically obvious puns about technology and poop—the latter of which are largely delivered by Patrick Stewart of all people, whose character Poop is once seen in a chair shouting “red alert”—and the delivery of every single one of these lines sounds so flat and bored that it’s clear that almost nobody cared to save the film from itself. The only one putting in any effort seems to be James Corden, whose vocal work is as bouncy as his character, but doesn’t have the timing to make any of his cringe-worthy lines work. If there’s any image that sums up the movie perfectly, it’s of that Just Dance avatar curled up and crying in the dump. It’s a horrifying scene set atop a pile of trash, but the pure absurdity of its existence is enough to make one laugh in shock and disbelief, at least in that one moment. The symbolism of that moment will stick with me far more than the stale and stilted messages The Emoji Movie intended.."
sentiment, probability = predict_sentiment(input_text)
print(f"\nInput: '{input_text}'")
print(f"Predicted Sentiment: {sentiment} (Probability: {probability:.4f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Input: 'The first two weeks of the Fall season are finally coming to a close, and most of the premieres have rolled out. We’ve still got a few reviews of pilots coming, before we turn to reviewing some of the returning shows, but I’ll be glad to rid my DVR of some of the more unwatchable premieres and settle into a nice weekly television schedule. Last night, the other half of ABC’s Wednesday night comedy block debuted (supplementing last week’s debuts of the more superior shows, “Modern Family” and to a much lesser extent, “Cougar Town”). The first, “Hank,” is an easy one to delete from the DVR — in fact, I nearly did so before the pilot had finished. It’s woefully, painfully unwatchable. The premise itself suits its star, Kelsey Grammar, but that is the only nice thing I can muster to say about “Hank.” It’s about a wealthy blowhard CEO in Manhattan who gets downsized and is forced to give up his nannies and maids and move to a much smaller, humbl